In [2]:
import numpy as np
import pandas as pd
import os
import random
from glob import glob
from sklearn.model_selection import train_test_split

In [3]:
"""
The dataset should be unzipped into the folder named 'data' This folder should be in the same directory as this file.
Extract the train.zip file.
"""
print(os.listdir("data/")) # should list train.zip, sample_submission... etc 
# The Most important folder is going to be the train folder. 
# We will create our validation set by splitting the train folder into train and valid folders.
if not os.path.exists('data/valid'):
    os.mkdir('data/valid/')

# Now we will create validation folders for each type of fish 
fish_types = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'] 
for fish in fish_types:
    # Creates a validation folder for each type of feature (fish)
    if not os.path.exists(f'data/valid/{fish}'):
        os.mkdir(f'data/valid/{fish}')

    # if valid folder already contains validation set, it breaks out of the loop
    if len(os.listdir(f'data/valid/{fish}')) > 0:
        print(f"Validation folder already contains files for {fish}")
        continue

    train_dir = f'data/train/{fish}/'
    valid_dir = f'data/valid/{fish}/'

    # List all files in the source directory
    file_paths = [os.path.join(train_dir, filename) for filename in os.listdir(train_dir)]

    # Use 20% of the files for validation, using the train_test_split function
    validation_ratio = 0.2
    train_files, valid_files = train_test_split(file_paths, test_size=validation_ratio, random_state=42)

    # Move the selected validation files to the validation directory
    for file_path in valid_files:
        filename = os.path.basename(file_path)
        destination = os.path.join(valid_dir, filename)
        os.rename(file_path, destination)

    print(f"{len(valid_files)} files from test/{fish} moved to validation set.")


# TODO
# Create a function to reset the validation, moving all validation images back into the test folder

['train.zip', 'sample_submission_stg1.csv', '__MACOSX', 'valid', 'sample_submission_stg1.csv.zip', 'test_stg2', 'sample_submission_stg2.csv.zip', 'test_stg2.7z', 'test_stg1.zip', 'train', 'test_stg1']
Validation folder already contains files for ALB
Validation folder already contains files for BET
Validation folder already contains files for DOL
Validation folder already contains files for LAG
Validation folder already contains files for NoF
Validation folder already contains files for OTHER
Validation folder already contains files for SHARK
Validation folder already contains files for YFT


In [4]:
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [5]:
""" 
This part of the code sets up an Image generator that will be used for the CNN
It doesn't actually compute any of the resizing or transformations yet, it just sets up the generator
The generator will be called when we train the model 
"""
# Create an ImageDataGenerator with data augmentation and normalization
datagen = ImageDataGenerator(
    rescale=1.0/255,    # Normalize pixel values to the range [0, 1]
    shear_range=0.2,    # Random shear transformations
    zoom_range=0.2,     # Random zooming
    horizontal_flip=True,  # Random horizontal flipping
)

# Load and preprocess the dataset (in this example, assume you have a directory of images)
train_generator = datagen.flow_from_directory(
    'data/train/',
    target_size=(224, 224),   # Resize images to a consistent size
    batch_size=32,            # Batch size for training
    class_mode='categorical'  # The type of labels (categorical for classification)
)

# Load and preprocess the validation set
validation_generator = datagen.flow_from_directory(
    'data/valid/',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

print(train_generator.class_indices)


Found 3019 images belonging to 8 classes.


Found 758 images belonging to 8 classes.
{'ALB': 0, 'BET': 1, 'DOL': 2, 'LAG': 3, 'NoF': 4, 'OTHER': 5, 'SHARK': 6, 'YFT': 7}


In [6]:
"""
This part of the code sets up the CNN model
"""
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(224, 224, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation='softmax'))

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Run the model
model.fit(
    train_generator,
    steps_per_epoch=10,
    epochs=1,
    validation_data=validation_generator,
    validation_steps=1
)

# Save the model
version = 1
model.save_weights(f'model{version}.h5')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 394272)            0         
                                                                 
 dense (Dense)               (None, 128)               50466944  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 8)                 1032      
                                                        

In [14]:
"""
Now run our trained model on the test set and create a submission csv file
"""
import keras
import sys

# Load the model 
model.load_weights(f'model{version}.h5')

# Load the first test set, test_stg1
# Submission must be a combination of test_stg1 (1000) and test_stg2 (12153) = 13153
test_set1 = glob('data/test_stg1/*.jpg')
test_set2 = glob('data/test_stg2/*.jpg')

# Each image name must be called 'test_stg1/image_000001.jpg', for example
for i, image in enumerate(test_set1):
    test_set1[i] = 'test_stg1/' + os.path.basename(image)
for i, image in enumerate(test_set2):
    test_set2[i] = 'test_stg2/' + os.path.basename(image)
test_set = test_set1 + test_set2
print(test_set[0:10])

# Create a dataframe to hold the predictions
submission = pd.DataFrame(columns=['image', 'ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT'])

# For each image in the test set, run the models prediction and save it to the submission df
# It takes about a min to run on test_stg1 for me (Karl). 
# It takes about 15 min to run the combined test_stg1 and test_stg2
# for each iteration, it takes 20 to 40ms to run. 
for i, image in enumerate(test_set):
    # Preprocess the test images
    test_image = test_set[i]
    test_image = tf.keras.preprocessing.image.load_img(test_image, target_size=(224, 224))
    test_image = tf.keras.preprocessing.image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    test_image = keras.applications.mobilenet.preprocess_input(test_image)

    # Run the prediction 
    prediction = model.predict(test_image)

    # round the prediction to the nearest tenth
    prediction = np.round(prediction, decimals=1)

    # print(f'prediction array: {prediction[0]}')

    # Get the predicted class
    # The prediction array is a list of probabilities for each class
    # For each prediction, map the prediction to the name of the class
    # train_generator.class_indices is a dictionary of the classes of fish and their indices
    keys = list(train_generator.class_indices.keys())

    # Create a dictionary of the predictions
    # Looks like this: 
    #  {'ALB': 1.3623509e-07, 'BET': 9.1926294e-20,
    #   'DOL': 2.9418256e-16, 'LAG': 1.9241103e-13, 'NoF': 0.0019608391, 
    #   'OTHER': 1.6155835e-11, 'SHARK': 2.625621e-13, 'YFT': 0.99803907}
    prediction_dict = dict(zip(keys, prediction[0]))
    # print(prediction_dict)

    # Add the prediction to the dataframe
    submission.loc[i, 'image'] = os.path.basename(test_set[i])
    for key in keys:
        submission.loc[i, key] = prediction_dict[key]

    # Uncomment the break if you want to test 
    # break


# Print the head, does it look okay?
print(submission.columns)
print(submission.head())

# Save the dataframe to a csv file :)
submission.to_csv('submission.csv', index=False)



['test_stg1/img_06658.jpg', 'test_stg1/img_06525.jpg', 'test_stg1/img_04860.jpg', 'test_stg1/img_00161.jpg', 'test_stg1/img_07623.jpg', 'test_stg1/img_02172.jpg', 'test_stg1/img_02993.jpg', 'test_stg1/img_05979.jpg', 'test_stg1/img_06475.jpg', 'test_stg1/img_01862.jpg']


SystemExit: 

/home/x1b3d3ad/.local/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
